# The forward and backward passes

In [1]:
from exp.matmul import *

In [2]:
#export
def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    
    return map(tensor, (x_train, y_train, x_valid, y_valid))


def normalize(x, mu, sig): return (x - mu) / sig

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_sig = x_train.mean(), x_train.std()

In [5]:
x_train = normalize(x_train, train_mean, train_sig)
x_valid = normalize(x_valid, train_mean, train_sig)

In [6]:
#export
def test_near_zero(a, tol=1e-3): assert a.abs() < tol, f'Not near-zero: {a}'

In [7]:
test_near_zero(x_train.mean())
test_near_zero(1 - x_train.std())

In [8]:
n, m = x_train.shape
c = y_train.max() + 1

n, m, c

(50000, 784, tensor(10))

# Foundations version

## Basic architecture

In [9]:
# num hidden
nh = 50

In [10]:
# he init
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [11]:
test_near_zero(w1.mean())
test_near_zero(w1.std() - 1 / math.sqrt(m))

In [12]:
# This should be ~ (0, 1)
x_valid.mean(), x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [13]:
def lin(x, w, b): return x @ w + b

In [14]:
t = lin(x_valid, w1, b1)

In [15]:
# ...so should this, because we used He init
t.mean(), t.std()

(tensor(0.0630), tensor(0.9588))

In [16]:
def relu(x): return x.clamp_min(0.)

In [17]:
t = relu(lin(x_valid, w1, b1))

In [18]:
# ... actually it really should be this!
t.mean(), t.std()

(tensor(0.4120), tensor(0.5752))

In [19]:
# Proper Kaiming He init
w1 = torch.randn(m, nh) * math.sqrt(2 / m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) * math.sqrt(2 / nh)
b2 = torch.zeros(1)

In [20]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5289), tensor(0.8172))

Much better.

It turns out that we could try instead:  

```{py}
def relu(x): return x.clamp_min(0.) - 0.5
```

and this works a little better.

In [21]:
#export
from torch.nn import init

In [22]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [23]:
t.mean(), t.std()

(tensor(0.5531), tensor(0.8271))

Why did we use `mode='fan_out'`? According to the docs, setting the variance to $\sqrt{\frac{2}{m}}$ gives unit variance in forward pass, while setting it to $\sqrt{\frac{2}{nh}}$ gives unit variance in backward pass. Now the default is te former, but we chose the latter. Why? Let's see how PyTorch creates layers:

In [24]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

It's the opposite! So how would the multiplication work?

In [25]:
torch.nn.Linear.forward??

Shows:

```{py}
@weak_script_method
    def forward(self, input):
        return F.linear(input, self.weight, self.bias)
```

In [26]:
torch.nn.functional.linear??

Shows:
    
```{py}
@weak_script
def linear(input, weight, bias=None):
    # type: (Tensor, Tensor, Optional[Tensor]) -> Tensor
    r"""
    Applies a linear transformation to the incoming data: :math:`y = xA^T + b`.

    Shape:

        - Input: :math:`(N, *, in\_features)` where `*` means any number of
          additional dimensions
        - Weight: :math:`(out\_features, in\_features)`
        - Bias: :math:`(out\_features)`
        - Output: :math:`(N, *, out\_features)`
    """
    if input.dim() == 2 and bias is not None:
        # fused op is marginally faster
        ret = torch.addmm(bias, input, weight.t())
    else:
        output = input.matmul(weight.t())  # ------> FOCUS HERE
        if bias is not None:
            output += bias
        ret = output
    return ret
```

So this is why we had to give the opposite parameter, because PyTorch does a transpose, and gets the same (784, 50) shape there.

So we could ask, what does PyTorch do for convolutional layers?

In [27]:
torch.nn.Conv2d??

So this is mostly documentation, and everything is passed to the superclass, `_ConvNd`.

In [28]:
torch.nn.modules.conv._ConvNd??

This basically just calls:  

```{py}
init.kaiming_uniform_(self.weight, a=math.sqrt(5))
```

while initializing weights, which is just He initialization, but it has a multiplier. This is not documented anywhere, and it seems to work badly.

In [29]:
def relu(x): return x.clamp_min(0.) - 0.5

In [30]:
w1 = torch.randn(m, nh) * math.sqrt(2 / m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) * math.sqrt(2 / nh)
b2 = torch.zeros(1)

In [31]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.0259), tensor(0.7779))

This also seems to get the variance near 1.

In [32]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [33]:
%timeit -n 10 model(x_valid)

16.8 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Loss function: MSE

In [34]:
model(x_valid).shape

torch.Size([10000, 1])

In [35]:
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [36]:
y_train, y_valid = y_train.float(), y_valid.float()

In [37]:
preds = model(x_train)

In [38]:
preds.shape

torch.Size([50000, 1])

In [39]:
mse(preds, y_train)

tensor(31.1138)

## Gradients and backward pass

In [40]:
def mse_grad(inp, targ):
    # Gradient of loss wrt output
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [41]:
def relu_grad(inp, out):
    # Gradient of relu wrt activations
    inp.g = (inp > 0).float() * out.g

In [42]:
def lin_grad(inp, out, w, b):
    # Gradient of matmul wrt input
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [43]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [44]:
%time forward_and_backward(x_train, y_train)

CPU times: user 3.79 s, sys: 3.62 s, total: 7.4 s
Wall time: 7.4 s


In [45]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We'll cheat a little and use PyTorch autograd to check our results

In [46]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [47]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [48]:
loss = forward(xt2, y_train)

In [49]:
loss.backward()

In [50]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Refactoring

## Classes as layers

In [51]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.) - 0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [52]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [53]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [54]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [55]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [56]:
%time loss = model(x_train, y_train)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 103 ms


In [57]:
%time model.backward()

CPU times: user 3.65 s, sys: 3.57 s, total: 7.22 s
Wall time: 7.22 s


In [58]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Module.forward()

There's some duplicate code among the layers, so let's fix that

In [59]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [60]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [61]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [62]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [63]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [64]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [65]:
%time loss = model(x_train, y_train)

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 100 ms


In [66]:
%time model.backward()

CPU times: user 196 ms, sys: 80 ms, total: 276 ms
Wall time: 275 ms


In [67]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

## Without einsum

In [68]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [69]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [70]:
%time loss = model(x_train, y_train)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 103 ms


In [71]:
%time model.backward()

CPU times: user 212 ms, sys: 64 ms, total: 276 ms
Wall time: 273 ms


In [72]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

Now that we've implemented these, we can use the PyTorch versions (that was our rule)

## nn.Linear and nn.Module

In [73]:
#export
from torch import nn

In [74]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [75]:
model = Model(m, nh, 1)

In [76]:
%time loss = model(x_train, y_train)

CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 102 ms


In [77]:
%time loss.backward()

CPU times: user 80 ms, sys: 0 ns, total: 80 ms
Wall time: 79.8 ms


So forward prop takes the same time, while backprop takes about a third, likely because PyTorch doesn't compute all the gradients and we did.

## Export

In [78]:
!python3.7 notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to fully.py
